In [33]:
!pip install -q langchain langgraph langchain-groq python-dotenv \
                langchain-community faiss-cpu sentence-transformers requests

In [34]:
import os
from dotenv import load_dotenv

load_dotenv()
assert os.getenv("GROQ_API_KEY") is not None, "❌ GROQ_API_KEY not loaded"
print("✅ GROQ API Key Loaded")

✅ GROQ API Key Loaded


In [35]:
from typing import Annotated, TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_core.tools import tool

In [36]:
llm_memory = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

In [37]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [38]:
def memory_chatbot(state: State):
    return {"messages": [llm_memory.invoke(state["messages"])]}

In [39]:
builder = StateGraph(State)
builder.add_node("chatbot", memory_chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

memory = MemorySaver()
memory_agent = builder.compile(checkpointer=memory)

print("✅ Memory Agent Ready")

✅ Memory Agent Ready


In [40]:
config = {"configurable": {"thread_id": "session_1"}}

memory_agent.invoke(
    {"messages": [("user", "My name is Sid.")]},
    config=config
)

result = memory_agent.invoke(
    {"messages": [("user", "What is my name?")]},
    config=config
)

print(result["messages"][-1].content)

Your name is Sid.


In [41]:
import requests

@tool
def get_weather(city: str):
    """Get current temperature of a city."""
    geo = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1, "format": "json"}
    ).json()

    if not geo.get("results"):
        return f"City {city} not found."

    lat = geo["results"][0]["latitude"]
    lon = geo["results"][0]["longitude"]

    weather = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={"latitude": lat, "longitude": lon, "current": "temperature_2m"}
    ).json()

    return f"The temperature in {city} is {weather['current']['temperature_2m']}°C"

In [42]:
llm_weather = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

llm_weather_tools = llm_weather.bind_tools([get_weather])

In [43]:
class WeatherState(TypedDict):
    messages: Annotated[list, add_messages]

def weather_chatbot(state: WeatherState):
    return {"messages": [llm_weather_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools=[get_weather])

def weather_router(state: WeatherState) -> Literal["tools", END]:
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

builder = StateGraph(WeatherState)
builder.add_node("chatbot", weather_chatbot)
builder.add_node("tools", tool_node)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", weather_router, {"tools": "tools", END: END})
builder.add_edge("tools", "chatbot")

weather_agent = builder.compile()
print("✅ Weather Agent Ready")

✅ Weather Agent Ready


In [44]:
weather_agent.invoke(
    {"messages": [("user", "What is the weather in London?")]}
)

{'messages': [HumanMessage(content='What is the weather in London?', additional_kwargs={}, response_metadata={}, id='17cbb3f1-0da4-4ba4-8ace-369941e88c4e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'y44eyhcqm', 'function': {'arguments': '{"city":"London"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 219, 'total_tokens': 233, 'completion_time': 0.025817704, 'completion_tokens_details': None, 'prompt_time': 0.013223285, 'prompt_tokens_details': None, 'queue_time': 0.055818755, 'total_time': 0.039040989}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b8a6f-7438-7022-a758-44fa07b8f550-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'London'}, 'id': 'y44eyhcqm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 219, 'output_t